In [ ]:
import numpy as np

# The predictions are in three separate numpy arrays.  Concatenate them into one.  
# We concatenate them on axis 0, as they only have one axis.
predictions = np.concatenate(predictions, axis=0)

# Map predictions to outcomes (only possible outcomes are 1 and 0)
predictions[predictions > .5] = 1
predictions[predictions <=.5] = 0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.asarray([0, 1, 2, 3, 4, 5])
# Let's set the slope of the line to 2.
y = 2 * x

# See how this line is "steeper" than before?  The larger the slope is, the steeper the line becomes.
# On the flipside, fractional slopes will create a "shallower" line.
# Negative slopes will create a line where y values decrease as x values increase.
plt.plot(x, y)
plt.show()

### Slope
The slope is multiplied by the x-value to get the new y value.
* y = mx
* m = slope

The equation to create the formula takes the form y=mx+b 

You might recognize pieces of this equation from the past two screens -- we're just adding the intercept and slope into one equation.

* This equation is saying "the predicted value of the second variable (y) is equal to the value of the first variable (x) times the slope (m) plus the intercept (b)".

We have to calculate values for mm and bb before we can use our formula.

We'll calculate slope first -- the formula is cov(x,y)/σ2x, which is just the covariance of x and y divided by the variance of x.

We can use the cov function to calculate covariance, and the .var() method on Pandas series to calculate variance.

In [ ]:
from numpy import cov

# This function will take in two columns of data, and return the slope of the linear regression line.
def calc_slope(x, y):
  return cov(x, y)[0, 1] / x.var()

intercept_density = wine_quality["quality"].mean() - (calc_slope(wine_quality["density"],
                                                                 wine_quality["quality"]) * wine_quality["density"].mean())

#### Scikit-Learn
To fit the model to the data, we'll use the machine learning library scikit-learn. Scikit-learn is the most popular library for working with machine learning models for small to medium sized datasets. Even when working with larger datasets that don't fit in memory, scikit-learn is commonly used to prototype and explore machine learning models on a subset of the larger dataset.

Scikit-learn uses an object-oriented style, so each machine learning model must be instantiated before it can be fit to a dataset (similar to creating a figure in Matplotlib before you plot values). We'll be working with the LinearRegression class from sklearn.linear_model:


from sklearn.linear_model import LinearRegression
lr = LinearRegression()
To fit a model to the data, we use the conveniently named fit method:


+ lr.fit(inputs, output)
>  where inputs is a matrix with n_samples rows and n_features columns. The output here can be in one of two formats:
   either an array with n_samples elements, for when you are predicting only one output, or
   a matrix with n_samples rows and n_outputs columns, for when you are predicting multiple outputs simultaneously.


The dataset we're working with contains 398 rows and 9 columns but since we want to only use the weight column, we need to pass in a matrix containing 398 rows and 1 column. The catch, however, is if you just select the weight column and pass that in as the first parameter to the fit method, an error will be returned. This is because scikit-learn will convert Series and Dataframe objects to NumPy objects and the dimensions won't match.

You can use the values attribute to see which NumPy object is returned:

+ cars["weight"].values


A NumPy array with 398 elements will be returned instead of a matrix containing rows and columns. You can confirm this by using the shape attribute:

+ cars["weight"].values.shape

The value (398,), representing 398 rows by 0 columns, will be returned. 


If you instead use double bracket notation:

+ cars[["weight"]].values

you'll get back a NumPy matrix with 398 rows and 1 column.

#### Error Metrics
The plot from the last step gave us a visual idea of how well the linear regression model performs. To obtain a more quantitative understanding, we can calculate the model's error, or the mismatch between a model's predictions and the actual values.

One commonly used error metric for regression is mean squared error, or MSE for short. You calculate MSE by computing the squared error between each predicted value and the actual value:

where Yi^Yi^ is a predicted value for fuel efficiency and YiYi is the actual mpg value. Then, you compute the mean of all of the squared errors:

Here's the same formula in psuedo-code:
sum = 0
for each data point:
    diff =  predicted_value - actual_value
    squared_diff = diff ** 2
    sum += squared_diff
    
mse = sum/n

We'll use the mean_squared_error function from scikit-learn to calculate MSE. We'll leave it to you to import the function and understand how to use it, so that you become more accustomed to reading documentation.

#### Root Mean Squared Error
There are many error metrics you can use, each with it's own advantages and disadvantages. While the specific properties of each of the different error metrics is outside the scope of this mission, we'll introduce another error metric here.

Root mean squared error, or RMSE for short, is the square root of the MSE and does a better job of penalizing large error values. In addition, the RMSE is easier to interpret since it's units are in the same dimension as the data. When computing MSE, we squared both the predicted and actual values, calculated the differences, then summed all of the differences. This means that the MSE value will be in miles per gallon squared while the RMSE value will be in miles per gallon.

#### Finding Error


Now that we know how to make a regression line manually, let's look at an easier way to do it, using a function from scipy.

The linregress function makes it simple to do linear regression.

Now that we know a simpler way to do linear regression, let's look at how to figure out if our regression is good or bad.

We can plot out our line and our actual values, and see how far apart they are on the y-axis.

We can also compute the distance between each prediction and the actual value -- these distances are called residuals.

If we add up the sum of the squared residuals, we can get a good error estimate for our line.

We have to add the squared residuals, because just like differences from the mean, the residuals add to 0 if they aren't squared.

To put it in math terms, the sum of squared residuals is: ∑i=1n(yi−y^i)2∑i=1n(yi−y^i)2

The variable



> cal

In [ ]:
from scipy.stats import linregress

# We've seen the r_value before -- we'll get to what p_value and stderr_slope are soon -- for now, don't worry about them.
slope, intercept, r_value, p_value, stderr_slope = linregress(wine_quality["density"], wine_quality["quality"])

# As you can see, these are the same values we calculated (except for slight rounding differences)
print(slope)
print(intercept)

y_predict = wine_quality["density"]*slope + intercept

rss = sum((y_predict - wine_quality["quality"])**2)

#### Standard Error
From the sum of squared residuals, we can find the standard error. The standard error is similar to the standard deviation, but it tries to make an estimate for the whole population of y-values -- even the ones we haven't seen yet that we may want to predict in the future.

The standard error lets us quickly determine how good or bad a linear model is at prediction.

The equation for standard error is √(RSS/n−2)

You take the sum of squared residuals, divide by the number of y-points minus two, and then take the square root.

You might be wondering about why 2 is subtracted -- this is due to differences between the whole population and a sample. This will be explained in more depth later on.